## Converting Ligand SMILE String to PDB

In [1]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem

In [2]:
df = pd.read_csv("compound.csv")

In [3]:
print(df)

              Names                                             Smiles
0         Quercetin   C1=CC(=C(C=C1C2=C(C(=O)C3=C(C=C(C=C3O2)O)O)O)O)O
1         Hypericin  CC1=CC(=O)C2=C(C3=C(C=C(C4=C3C5=C2C1=C6C(=CC(=...
2  Chlorogenic acid  C1[C@H]([C@H]([C@@H](C[C@@]1(C(=O)O)O)OC(=O)/C...
3        Kaempferol      C1=CC(=CC=C1C2=C(C(=O)C3=C(C=C(C=C3O2)O)O)O)O
4       Epicatechin  C1[C@H]([C@H](OC2=CC(=CC(=C21)O)O)C3=CC(=C(C=C...


In [4]:
print(df.columns)

Index(['Names', 'Smiles'], dtype='object')


In [6]:
for i, row in df.iterrows():
    mol = Chem.MolFromSmiles(row["Smiles"])
    mol = Chem.AddHs(mol)  # Add hydrogen atoms
    AllChem.EmbedMolecule(mol, AllChem.ETKDG())  # Generate 3D conformation
    Chem.MolToPDBFile(mol, f"{row['Names']}.pdb")
    print(f"Saved: {row['Names']}.pdb")

Saved: Quercetin.pdb
Saved: Hypericin.pdb
Saved: Chlorogenic acid.pdb
Saved: Kaempferol.pdb
Saved: Epicatechin.pdb


## Convert to PDBQT

In [7]:
import os
import glob

In [8]:
# Create "ligands" folder if it doesn't exist
output_folder = "ligands"
os.makedirs(output_folder, exist_ok=True)

In [11]:
pdb_files = glob.glob("*.pdb")

for pdb_file in pdb_files:
    pdbqt_file = os.path.join(output_folder, pdb_file.replace(".pdb", ".pdbqt"))
    
    # Open Babel command to convert PDB to PDBQT
    command1 = f"obabel {pdb_file} -O temp_3d.pdb --gen3d"
    os.system(command1)

    command2 = f"obabel temp_3d.pdb -O {pdbqt_file} --partialcharge gasteiger"
    os.system(command2)
    
    print(f"Converted {pdb_file} to {pdbqt_file}")

Converted 3poz.pdb to ligands\3poz.pdbqt
Converted Chlorogenic_acid.pdb to ligands\Chlorogenic_acid.pdbqt
Converted Epicatechin.pdb to ligands\Epicatechin.pdbqt
Converted Hypericin.pdb to ligands\Hypericin.pdbqt
Converted Kaempferol.pdb to ligands\Kaempferol.pdbqt
Converted Quercetin.pdb to ligands\Quercetin.pdbqt
Converted temp_3d.pdb to ligands\temp_3d.pdbqt


# Convert and Cleaning Receptor EPQR

In [38]:
import os
from Bio import PDB
import subprocess

In [41]:
output_folder = "receptor"
os.makedirs(output_folder, exist_ok=True)

cleaned_pdb = os.path.join(output_folder, "3POZ_cleaned.pdb")
pdbqt_file = os.path.join(output_folder, "3POZ_cleaned.pdbqt")

parser = PDB.PDBParser(QUIET=True)
structure = parser.get_structure("Receptor", input_pdb)

## Remove Ligands (03P1023), SO4, and Water

In [43]:
for model in structure:
    for chain in model:
        residues_to_keep = []
        for residue in chain:
            hetflag = residue.id[0]  # Check if it's a heteroatom
            resname = residue.resname

           # Remove ligands, SO4, and water)
            if hetflag == " " and "CA" in residue:  
                residues_to_keep.append(residue)
        
        chain.child_list = residues_to_keep

### To save the Cleaned file

In [44]:
io = PDB.PDBIO()
io.set_structure(structure)
io.save(cleaned_pdb)
print(f"Cleaned receptor saved as: {cleaned_pdb}")

Cleaned receptor saved as: receptor\3POZ_cleaned.pdb


### Add Polar hydrogen and Gasteiger charges

In [45]:
command_h = f"obabel {cleaned_pdb} -O {cleaned_pdb} --addpolarh"
subprocess.run(command_h, shell=True)

# To Assign Gasteiger Charges
command_charge = f"obabel {cleaned_pdb} -O {cleaned_pdb} --partialcharge gasteiger"
subprocess.run(command_charge, shell=True)


CompletedProcess(args='obabel receptor\\3POZ_cleaned.pdb -O receptor\\3POZ_cleaned.pdb --partialcharge gasteiger', returncode=0)

### Convert to PDBQT

In [50]:
command = f"obabel {cleaned_pdb} -O {pdbqt_file}"
subprocess.run(command, shell=True)

print(f"Receptor converted to PDBQT and saved as: {pdbqt_file}")

Receptor converted to PDBQT and saved as: receptor\3POZ_cleaned.pdbqt
